In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)

CODIGO_HOSPITAL = 112102

En este cuadernillo se quieren identificar los diagósticos más relevantes productivamente y nacionlamente del HLCM.

In [5]:
df = pd.read_csv("../data/interim/ranking_nacional_egresos.csv", encoding="latin-1", sep=";")